In [1]:
### html browser

In [1]:
# from flask import Flask, render_template 
# app=Flask(__name__) 
# @app.route("/") 
# def home(): 
#     return render_template("../src/home.html") 
# if __name__ =="__main__": 
#     app.run(debug=True) 

In [170]:
def remove_whitespace_entities(doc):
    doc.ents = [e for e in doc.ents if not e.text.isspace()]
    return doc

nlp.add_pipe(remove_whitespace_entities, after='ner')
doc = nlp(u'Hello\nNew York')
print(doc.ents)
# (New York,)

(New York,)


In [171]:
import pandas as pd
import numpy as np
import spacy
import random
from spacy.util import minibatch, compounding

nlp = spacy.load('en_core_web_lg')


In [172]:
df = pd.read_csv("../datawe/raw/Email_Classification/small_train.csv")
entityDf = df[df.apply(lambda x: x["text"][x["start_char"]:x["end_char"]] == x["Entity"], axis=1)]

In [180]:
entityTrainData = []
for text, item in  entityDf['text'].value_counts().items():
#     print(item, text)
    mulItems = entityDf[entityDf['text'] == text]
    multipleEntities = []
    for dta in mulItems.values:
#         print(dta)
        multipleEntities.append((dta[4], dta[2], dta[3]))
#          entityTrainData.append((dta[5], ))
    entityTrainData.append((text, {'entities':multipleEntities}))
    
TRAIN_DATA = entityTrainData

In [181]:
# TRAIN_DATA

In [182]:
# entityDf

In [195]:
def main_train(model=None, output_dir=None, n_iter=120):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print('Losses', losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
#         print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#         print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)

        # test the saved model
#         print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
#             print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#             print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    return nlp



In [196]:
trainedModel = main_train('en_core_web_sm')

Loaded model 'en_core_web_sm'
Losses {'ner': 3.315060920795562}
Losses {'ner': 3.404045535647962}
Losses {'ner': 3.930275830262143}
Losses {'ner': 3.06657715512887}
Losses {'ner': 2.7897050859087575}
Losses {'ner': 3.4477013516382}
Losses {'ner': 2.856652928588005}
Losses {'ner': 2.198723471604673}
Losses {'ner': 3.9322881182655935}
Losses {'ner': 2.0247766211214935}
Losses {'ner': 0.9683140641037671}
Losses {'ner': 0.9164910951901817}
Losses {'ner': 2.3304153550250812}
Losses {'ner': 1.0981210743502539}
Losses {'ner': 1.1953628250814463}
Losses {'ner': 0.8796425276903744}
Losses {'ner': 0.9564163050529851}
Losses {'ner': 1.242199325988068}
Losses {'ner': 1.2970445863195614}
Losses {'ner': 1.3454837944351492}
Losses {'ner': 1.5739944373588446}
Losses {'ner': 1.2971613946319005}
Losses {'ner': 0.5468275043892331}
Losses {'ner': 0.5687733131732137}
Losses {'ner': 0.29721300995993744}
Losses {'ner': 0.5135165650669608}
Losses {'ner': 0.548890644508621}
Losses {'ner': 0.3721958532234902}
L

In [192]:
TRAIN_DATA

[('check in and check out date 31/12/2018, 1/1/2019',
  {'entities': [(28, 38, 'DATE'), (40, 48, 'DATE')]}),
 ('i want to book a room at lake palace. Check in date is - 31st December 2018, check out - 1st January 2019',
  {'entities': [(25, 36, 'LOC'), (57, 75, 'DATE')]}),
 ('Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
  {'entities': [(5, 10, 'PERSON'),
    (96, 114, 'ORG'),
    (120, 139, 'DATE'),
    (144, 154, 'DATE')]}),
 ('i want to book a room at lake palace. Check in date is - 20th december 2018, check out - 8th January 2019',
  {'entities': [(89, 105, 'DATE')]}),
 ('As per corporate holiday plan, kindly arrange to book the following accommodation in your Hotel/Resort as per details given below: Serial number : TEHP/18/12809 Name : Arun Gadamshetty P No. : 153021 Department : Spares Manufacturing Department ',
  {'entities': [(90, 102, 'FAC'),
    (147, 160, 'ORG'),


In [92]:
# import spacy

# nlp = spacy.load('en_core_web_lg')
multipleLines = """i want to book a room at Lake palace.
Check in date is - 31/12/2018
check out - 1/1/2019"""
# nlp = spacy.load('en_core_web_lg')
# doc = trainedModel(u''+entityDf["text"][13])
doc = trainedModel(u''+ " ".join(entityDf["text"]))
trn_data = []
for ent in doc.ents:
#     trn_data.append({Entity, end_char,label,start_char, text})
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Rohit 5 10 PERSON
Taj Gateway Ganges 96 114 ORG
24/26 December 2018 120 139 DATE
two nights 144 154 DATE
Rohit 161 166 PERSON
Taj Gateway Ganges 252 270 ORG
24/26 December 2018 276 295 DATE
two nights 300 310 DATE
Rohit 317 322 PERSON
Taj Gateway Ganges 408 426 ORG
24/26 December 2018 432 451 DATE
two nights 456 466 DATE
Rohit 473 478 PERSON
Taj Gateway Ganges 564 582 ORG
24/26 December 2018 588 607 DATE
two nights 612 622 DATE
Agnes 627 632 PERSON
Hotel/Resort 779 791 FAC
TEHP/18/12809 836 849 ORG
Arun Gadamshetty 857 873 PERSON
153021 882 888 CARDINAL
Department : Spares Manufacturing Department 889 933 ORG
Hotel/Resort 1025 1037 FAC
TEHP/18/12809 1082 1095 ORG
Arun Gadamshetty 1103 1119 PERSON
153021 1128 1134 CARDINAL
Department : Spares Manufacturing Department 1135 1179 ORG
Hotel/Resort 1271 1283 FAC
TEHP/18/12809 1328 1341 ORG
Arun Gadamshetty 1349 1365 PERSON
153021 1374 1380 CARDINAL
Department : Spares Manufacturing Department 1381 1425 ORG
Hotel/Resort 1517 1529 FAC
TEHP/18/

In [194]:
# " ".join(entityDf["text"])
doc = trainedModel(u''+ """i want to book a room at Lake palace Check in date is - 31st December 2018 
check out - 1st January 2019""")
trn_data = []
for ent in doc.ents:
#     trn_data.append({Entity, end_char,label,start_char, text})
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Lake palace 25 36 LOC
31st December 2018 56 74 DATE


In [1]:
sentence = ["i want to book a room at Lake palace. Check in date is - 31st December 2018, check out - 1st January 2019"]

In [42]:
dte1 = "31/12/2018"
dte2 = "1/1/2019"
tetx = "check in and check out date 31/12/2018, 1/1/2019"
tetx.index(dte1), + tetx.index(dte1) + len(dte1)

(28, 38)

In [43]:
tetx.index(dte2), + tetx.index(dte2) + len(dte2)

(40, 48)

In [7]:
date1 ="31st December 2018"
loc = "Lake palace"
date2 ="1st January 2019"
sentence[0].index(date1), + sentence[0].index(date1) + len(date1)

(57, 75)

In [8]:
sentence[0].index(date2), + sentence[0].index(date2) + len(date2)

(89, 105)

In [35]:
entityDf["text"][13]

'i want to book a room at Lake palace. Check in date is - 31st December 2018, check out - 1st January 2019'

In [77]:
trainedModel.get_pipe('textcat')

KeyError: "[E001] No component 'textcat' found in pipeline. Available names: ['tagger', 'parser', 'ner']"

In [126]:
palace = nlp.vocab[u'palace']
Palace = nlp.vocab[u'Palace']
palace.is_lower = Palace.is_lower
palace.shape = Palace.shape
palace.is_upper = Palace.is_upper
palace.cluster = Palace.cluster

In [187]:
### Testing

doc = trainedModel(u''+ "check in and check out date 31/12/2018, 1/1/2019")
trn_data = []
for ent in doc.ents:
#     trn_data.append({Entity, end_char,label,start_char, text})
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

31/12/2018 28 38 DATE
1/1/2019 40 48 DATE


In [113]:
nlp.vocab[u'india']

In [127]:
nlp(u'palace').ents[0]

IndexError: tuple index out of range

In [119]:
doc = nlp(u'south korea is a state in asia.')
for word in doc:
    print(word.text, word.tag_, word.ent_type_)

south JJ 
korea NN GPE
is VBZ 
a DT 
state NN 
in IN 
asia NN 
. . 


In [120]:
probs = {w.prob: w.orth for w in nlp.vocab}
usually_titled = [w for w in nlp.vocab if w.is_title and probs.get(w.lower, -10000) < probs.get(w.orth, -10000)]

In [121]:
usually_titled

[]

In [131]:
# probs

In [138]:
from spacy.attrs import ORTH, LEMMA, TAG, ENT_TYPE, ENT_IOB
nlp = spacy.load('en_core_web_lg')

In [151]:
nlp.tokenizer.add_special_case('south korea', [{ORTH: 'south korea', LEMMA: 'South korea', TAG: 'NNP', ENT_TYPE: 'GPE', ENT_IOB: 3}])
doc = nlp(u'there are many innovative companies in south korea.')
[(w.text, w.label_) for w in doc.ents]

[('south', 'GPE'), ('korea', 'GPE')]

In [141]:
nlp.tokenizer.add_special_case('india', [{ORTH: 'india', LEMMA: 'India', TAG: 'NNP', ENT_TYPE: 'GPE', ENT_IOB: 3}])
doc = nlp(u'there are many innovative companies in india.')
[(w.text, w.tag_, w.ent_type_) for w in doc]

[('there', 'EX', ''),
 ('are', 'VBP', ''),
 ('many', 'JJ', ''),
 ('innovative', 'JJ', ''),
 ('companies', 'NNS', ''),
 ('in', 'IN', ''),
 ('india', 'NNP', 'GPE'),
 ('.', '.', '')]

In [146]:
doc = nlp(u'south korea is a state in asia.')
for word in doc.ents:
    print(word.text, word.label_)

korea GPE
asia GPE


In [153]:
df.to_csv("../datawe/raw/Email_Classification/small_train.csv")

In [169]:
from spacy.attrs import ORTH, LEMMA, TAG, ENT_TYPE, ENT_IOB
# nlp = spacy.load('en')
nlp.tokenizer.add_special_case('taj', [{ORTH: 'taj', LEMMA: 'Taj', TAG: 'NNP', ENT_TYPE: 'FAC', ENT_IOB: 3}])
nlp.tokenizer.add_special_case('lake', [{ORTH: 'lake', LEMMA: 'Lake', TAG: 'NNP', ENT_TYPE: 'FAC', ENT_IOB: 3}])
nlp.tokenizer.add_special_case('pallace', [{ORTH: 'pallace', LEMMA: 'Pallace', TAG: 'NNP', ENT_TYPE: 'FAC', ENT_IOB: 3}])

doc = nlp(u'there are many innovative companies in taj lake pallace .')
[(w.text, w.label_) for w in doc.ents]

[('taj', 'FAC'), ('lake', 'FAC'), ('pallace', 'FAC')]